# Lasso Regression From Scratch

Lasso (Least Absolute Shrinkage and Selection Operator) adds an **L1 penalty** to the cost function. Unlike Ridge, Lasso can shrink coefficients exactly to zero, performing automated **feature selection**.

## Key Concepts:
- **L1 Regularization**: Penalty of $\alpha \sum |w_j|$
- **Coordinate Descent**: Standard iterative algorithm for Lasso
- **Soft Thresholding**: The operator used to update weights
- **Sparsity**: Produces sparse models (many zero weights)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso as SklearnLasso

## 1. Mathematical Foundation

### Cost Function
$$J(w, b) = \frac{1}{2n} \sum_{i=1}^n (y_i - (X_i w + b))^2 + \alpha \sum_{j=1}^m |w_j|$$

### Coordinate Descent Update
For each feature $j$:
1. Calculate the partial fit without feature $j$: $\rho_j = \sum_{i=1}^n x_{ij} (y_i - (\sum_{k \neq j} x_{ik} w_k + b))$
2. Normalize the feature: $z_j = \sum_{i=1}^n x_{ij}^2$
3. Apply **Soft Thresholding**:
   $$w_j = S(\rho_j, \alpha \cdot n) / z_j$$
   Where $S(z, \gamma) = \text{sign}(z) \max(|z| - \gamma, 0)$

In [ ]:
class LassoRegression:
    def __init__(self, alpha=1.0, max_iter=1000, tol=1e-4):
        self.alpha = alpha
        self.max_iter = max_iter
        self.tol = tol
        self.w = None
        self.b = 0

    def _soft_threshold(self, rho, lam):
        if rho < -lam:
            return rho + lam
        elif rho > lam:
            return rho - lam
        else:
            return 0

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = np.mean(y)
        
        for _ in range(self.max_iter):
            w_old = self.w.copy()
            
            for j in range(n_features):
                # Calculate rho_j: sum(x_ij * (y_i - y_pred_without_j))
                y_pred = X @ self.w + self.b
                rho = np.dot(X[:, j], y - y_pred + self.w[j] * X[:, j])
                
                # Update weight using soft thresholding
                # Note: alpha is scaled by n_samples in many implementations
                self.w[j] = self._soft_threshold(rho, self.alpha * n_samples) / np.sum(X[:, j]**2)
            
            # Update bias
            self.b = np.mean(y - X @ self.w)
            
            # Convergence check
            if np.linalg.norm(self.w - w_old) < self.tol:
                break
                
        return self

    def predict(self, X):
        return X @ self.w + self.b

    def score(self, X, y):
        y_pred = self.predict(X)
        return 1 - np.sum((y - y_pred)**2) / np.sum((y - np.mean(y))**2)

## 2. Comparison with Sklearn

We use feature scaling as Lasso is sensitive to feature magnitude.

In [ ]:
X, y = make_regression(n_samples=100, n_features=10, n_informative=3, noise=10, random_state=42)
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

alpha = 1.0

lasso = LassoRegression(alpha=alpha)
lasso.fit(X_train, y_train)

sk_lasso = SklearnLasso(alpha=alpha)
sk_lasso.fit(X_train, y_train)

print(f"Our Lasso R2: {lasso.score(X_test, y_test):.4f}")
print(f"Sklearn Lasso R2: {sk_lasso.score(X_test, y_test):.4f}")

print("\nCoefficients Comparison (first 5):")
print(f"Our: {lasso.w[:5]}")
print(f"SK:  {sk_lasso.coef_[:5]}")

## 3. Visualizing Feature Selection

Watch how coefficients go to zero as alpha increases.

In [ ]:
alphas = [1e-3, 0.1, 1, 10, 100]
coeff_matrix = []

for a in alphas:
    l = LassoRegression(alpha=a)
    l.fit(X_train, y_train)
    coeff_matrix.append(l.w)

plt.figure(figsize=(10, 6))
plt.plot(alphas, coeff_matrix)
plt.xscale('log')
plt.xlabel('Alpha')
plt.ylabel('Coefficients Value')
plt.title('Lasso Paths - Feature Selection')
plt.grid(True)
plt.show()